In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np

2023-06-08 11:51:59.491806: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Clear any logs from previous runs
import shutil
shutil.rmtree('logs')

## Initial setup

- Import VGG16 pretrained model

In [3]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

batch_size = 16
img_height = 184
img_width = 216
img_size = (img_height, img_width)
img_shape = img_size + (3,)

with tf.device("CPU"):
    pre_trained_model = VGG16(input_shape = img_shape,
                            include_top = False,
                            weights = 'imagenet')

    pre_trained_model.trainable = True
print("Number of layers: ", len(pre_trained_model.layers))

2023-06-08 11:52:01.769941: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 11:52:02.363461: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1637] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7910 MB memory:  -> device: 0, name: A100-SXM4-40GB MIG 2g.10gb, pci bus id: 0000:90:00.0, compute capability: 8.0


Number of layers:  19


- Load dataset

In [4]:
# Edit according to local path for dataset
ds_path = r"/drive0-storage/Gracia/dataset"

with tf.device("CPU"):
    train_ds = tf.keras.utils.image_dataset_from_directory(ds_path,
                                            validation_split = 0.2,
                                            subset = "training",
                                            seed = 123,
                                            image_size = img_size,
                                            batch_size = batch_size)

    val_ds = tf.keras.utils.image_dataset_from_directory(ds_path,
                                          validation_split = 0.2,
                                          subset = "validation",
                                          seed = 123,
                                          image_size = img_size,
                                          batch_size = batch_size)

class_names = train_ds.class_names
num_classes = len(class_names)

Found 21413 files belonging to 3 classes.
Using 17131 files for training.
Found 21413 files belonging to 3 classes.
Using 4282 files for validation.


* Hyperparameter setup

In [5]:
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-5, 1e-4, 1e-3, 1e-2]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.4))
# HP_TUNING_LAYER = hp.HParam('tuning_layers', hp.IntInterval(165, 170))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_LEARNING_RATE, HP_DROPOUT],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

## Define training & run function

Optimizer Adam

In [8]:
def model_training(hparams):
    # Averaging layer
    global_average = tf.keras.layers.GlobalAveragePooling2D()

    # Add dense layer
    prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

    # Chain model 
    inputs = tf.keras.Input(shape = img_shape)
    x = preprocess_input(inputs)
    x = pre_trained_model(x, training=False)
    x = global_average(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs,outputs)
    
    # Compile model
    LR = hparams[HP_LEARNING_RATE]
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                  metrics=['accuracy'])
    
    # Train model
    EPOCH = 15

    model.fit(train_ds,
        validation_data = val_ds,
        epochs = EPOCH,
        callbacks=[
            tf.keras.callbacks.TensorBoard(logdir),  
            hp.KerasCallback(logdir, hparams)
            ]
        )
    _, accuracy = model.evaluate(val_ds)
    return accuracy

Optimizer SGD

In [ ]:
def model_training_SGD(hparams):
    # Averaging layer
    global_average = tf.keras.layers.GlobalAveragePooling2D()

    # Add dense layer
    prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

    # Chain model 
    inputs = tf.keras.Input(shape = img_shape)
    x = preprocess_input(inputs)
    x = pre_trained_model(x, training=False)
    x = global_average(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs,outputs)
    
    # Compile model
    LR = hparams[HP_LEARNING_RATE]
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate = LR),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                  metrics=['accuracy'])
    
    # Train model
    EPOCH = 15

    model.fit(train_ds,
        validation_data = val_ds,
        epochs = EPOCH,
        callbacks=[
            tf.keras.callbacks.TensorBoard(logdir),  
            hp.KerasCallback(logdir, hparams)
            ]
        )
    _, accuracy = model.evaluate(val_ds)
    return accuracy

## Start runs

In [9]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = model_training(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0
logdir = r'logs/hparam_tuning'

for learning_rate in HP_LEARNING_RATE.domain.values:
    for dropout_rate in np.linspace(HP_DROPOUT.domain.min_value,
                                    HP_DROPOUT.domain.max_value,
                                    3):
#         for tuning_layer in (HP_TUNING_LAYER.domain.min_value,
#                              HP_TUNING_LAYER.domain.max_value):
#                             np.linspace(HP_TUNING_LAYER.domain.min_value,
#                                         HP_TUNING_LAYER.domain.max_value,
#                                         6, dtype='int'):
        hparams = {
                  HP_LEARNING_RATE: learning_rate,
                  HP_DROPOUT: dropout_rate
#                   HP_TUNING_LAYER: tuning_layer,
                  }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1

--- Starting trial: run-0
{'learning_rate': 1e-05, 'dropout': 0.2}
Epoch 1/15


2023-06-08 11:54:18.891369: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-08 11:54:19.962444: I tensorflow/stream_executor/cuda/cuda_blas.cc:1633] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1071/1071 [==============================] - 115s 104ms/step - loss: 0.9781 - accuracy: 0.5575 - val_loss: 0.8929 - val_accuracy: 0.5831
Epoch 2/15
1071/1071 [==============================] - 110s 103ms/step - loss: 0.8075 - accuracy: 0.6367 - val_loss: 0.7203 - val_accuracy: 0.6714
Epoch 3/15
1071/1071 [==============================] - 110s 102ms/step - loss: 0.6010 - accuracy: 0.7468 - val_loss: 0.6153 - val_accuracy: 0.7506
Epoch 4/15
1071/1071 [==============================] - 110s 102ms/step - loss: 0.3752 - accuracy: 0.8518 - val_loss: 0.4165 - val_accuracy: 0.8375
Epoch 5/15
1071/1071 [==============================] - 110s 102ms/step - loss: 0.2197 - accuracy: 0.9183 - val_loss: 0.3249 - val_accuracy: 0.8860
Epoch 6/15
1071/1071 [==============================] - 110s 102ms/step - loss: 0.1293 - accuracy: 0.9552 - val_loss: 0.2671 - val_accuracy: 0.8989
Epoch 7/15
1071/1071 [==============================] - 109s 102ms/step - loss: 0.0907 - accuracy: 0.9681 - val_loss: 0.157

## Visualize in TensorBoard

In [41]:
%tensorboard --logdir logs/hparam_tuning